# Protein Folding | EPOCH Lab 2022

In [1]:
%matplotlib inline

from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

import nglview as nv

In [2]:
try:
  platform = Platform.getPlatformByName("CUDA")
except Exception:
  platform = Platform.getPlatformByName("OpenCL")

In [11]:
pdb = PDBFile('pdb/input.pdb')
forcefield = ForceField('amber99sb.xml', 'tip3p.xml')
# forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

In [12]:
modeller = Modeller(pdb.topology, pdb.positions)
# modeller.addHydrogens(forcefield)
# modeller.addExtraParticles(forcefield)
modeller.topology

<Topology; 1 chains, 2798 residues, 8867 atoms, 6111 bonds>

In [13]:
unmatched_residues = forcefield.getUnmatchedResidues(pdb.topology)
unmatched_residues

[]

In [14]:
system = forcefield.createSystem(modeller.topology,
                                 nonbondedMethod=PME,
                                 nonbondedCutoff=1*nanometer,
                                 constraints=HBonds)

In [15]:
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)

In [16]:
simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('tmp/output.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True, potentialEnergy=True, temperature=True))

In [17]:
simulation.step(10000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-123574.78641541512,251.7548419395362
2000,-118079.72391541512,289.046958705751
3000,-116276.66141541512,298.66109638191637
4000,-115585.84891541512,306.7239644240748
5000,-115697.75516541512,303.3229369849745
6000,-115390.06766541512,302.5961171756357
7000,-115567.09891541512,300.0767836531827
8000,-115848.97391541512,305.36089048544824
9000,-115537.53641541512,295.1248893301636
10000,-115360.09891541512,296.4517122644438


In [21]:
view = nv.show_file('tmp/output.pdb')
view

NGLWidget()